In [1]:
import sys
sys.path.append('../src')

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import wavy

/Users/ibiscp/GitHub/logspace/wavy/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [7]:
msft = yf.Ticker("MSFT")

# get stock info
msft.info

# get historical market data from 2005 on
hist = msft.history(period="max", start="2005-01-01")

In [8]:
hist = hist.pct_change()

In [ ]:
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2005-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,0.002611,5.565318e-03,0.000375,0.003739,0.683649,NaN,NaN
2005-01-05,-0.001116,5.690703e-07,0.003752,-0.002235,-0.337883,NaN,NaN
2005-01-06,0.000373,-1.476029e-03,-0.004484,-0.001120,0.061093,NaN,NaN
2005-01-07,-0.001117,-6.282191e-03,-0.000751,-0.002991,-0.106219,NaN,NaN


In [7]:
# Add column with random boolean values
hist["random"] = np.random.randint(0, 2, size=len(hist))
hist['random'] = hist['random'].astype(bool)

In [7]:
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2005-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,0.002611,5.565318e-03,0.000375,0.003739,0.683649,NaN,NaN
2005-01-05,-0.001116,5.690703e-07,0.003752,-0.002235,-0.337883,NaN,NaN
2005-01-06,0.000373,-1.476029e-03,-0.004484,-0.001120,0.061093,NaN,NaN
2005-01-07,-0.001117,-6.282191e-03,-0.000751,-0.002991,-0.106219,NaN,NaN


In [9]:
hist.drop(columns=["Dividends", "Stock Splits"], inplace=True)

In [10]:
hist.dropna(inplace=True)

In [10]:
hist.shape

(4379, 5)

In [11]:
x, y = wavy.create_panels(hist, lookback=10, horizon=1)

In [12]:
x.shape[1]

10

In [13]:
# predict only the highest price
y = y[['High']]

In [14]:
from wavy import models

In [20]:
baseline = models.ConvModel(x, y, kernel_size=10, model_type="regression")

2022-05-26 21:00:28.566633: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:

baseline.fit() # val_MAE: 0.0136

99/99 [==============================] - 1s 3ms/step - loss: 8.3809e-04 - MAE: 0.0191 - val_loss: 3.0108e-04 - val_MAE: 0.0128


In [22]:
baseline.score(on="train")

loss    0.000329
MAE     0.013082
dtype: float64

In [22]:
x.shape

(4369, 10, 5)

In [44]:
a = baseline.predict()

In [45]:
a[0]

,High
Date,
2005-01-19,0.013395


In [31]:
c = Panel([pd.DataFrame(b, columns=y[0].columns) for b in a])

In [34]:
c[1]

,High
0,0.02106


In [25]:
from wavy import Panel
b = Panel(a)

In [27]:
b[0]

array([[0.01339532]], dtype=float32)

In [21]:
a.shape

(4369, 1, 1)

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

linearRegression = models.ShallowModel(x, y, model=LinearRegression, metrics=[mean_absolute_error])

In [16]:
linearRegression.fit()

LinearRegression()

In [17]:
a = linearRegression.predict()

In [18]:
y[0]

,High
Date,
2005-01-19,-0.003415


In [19]:
a[0]

,High
Date,
2005-01-19,0.006219


In [ ]:
linearRegression.score(on="train")

In [23]:
metrics = [mean_absolute_error]

for a in metrics:
    a(linearRegression, linearRegression.x_train, linearRegression.y_train)

TypeError: mean_absolute_error() takes 2 positional arguments but 3 were given

## Autoconvert bool to int

In [ ]:
a = x[0].copy()

In [ ]:
a['random'].dtype

In [ ]:
a['random'] = a['random'].astype(int)

In [ ]:
a['random'].dtype

In [ ]:
for col in x[0].columns:
    if x[0][col].dtype != np.float64 or x[0][col].dtype != np.int:
        raise ValueError(f"Column {col} is not numeric.")

In [ ]:
y[0]

In [ ]:
for i in y:
    for col in i.columns:
        # Convert columnn to int if bool
        if i[col].dtype == bool:
            i[col] = i[col].astype(int)

In [ ]:
y[2]